Initialize the source system table.

In [ ]:
IF (NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dbo' AND  TABLE_NAME = 'SourceSystem'))
BEGIN
	CREATE TABLE [dbo].[SourceSystem] (
		[source_system_id] [int] NOT NULL,
		[system_name] [nvarchar](32) NULL,
		[system_version] [nvarchar](16) NULL,
		[system_type] [nvarchar](32) NULL,
		[external_schema] [nvarchar](16) NULL,
		[is_production] [bit] NOT NULL,
		CONSTRAINT [PK_SourceSystem] PRIMARY KEY CLUSTERED ( [source_system_id] ASC )
	);
END

-- Set up temp table
IF OBJECT_ID('tempdb..#SourceDatabases') IS NOT NULL
	DROP TABLE #SourceDatabases;

SELECT TOP(0) *
INTO #SourceDatabases 
FROM [dbo].[SourceSystem];

-- Add table info
INSERT INTO #SourceDatabases (
	  source_system_id
	, system_name
	, system_version
	, system_type
	, external_schema
	, is_production
)
VALUES
	(0, 'Unknown', NULL, NULL, NULL, 0),
	(1, 'ERP Name', '1', 'ERP', 'erp', 1),
	(2, 'System 1', '10', null, 'sys1', 0),
	(3, 'Salesforce', NULL, 'CRM', 'sf', 1);

-- Load Data
MERGE [dbo].[SourceSystem] AS target
USING (
	SELECT 
		  source_system_id
		, system_name
		, system_version
		, system_type
		, external_schema
		, is_production
	FROM #SourceDatabases
) AS source
ON (target.source_system_id = source.source_system_id)
WHEN MATCHED THEN
	UPDATE SET 
		  target.system_name = source.system_name
		, target.system_version = source.system_version
		, target.system_type = source.system_type
		, target.external_schema = source.external_schema
		, target.is_production = source.is_production
WHEN NOT MATCHED THEN
	INSERT (
		  source_system_id
		, system_name
		, system_version
		, system_type
		, external_schema
		, is_production
	)
	VALUES (
		  source.source_system_id
		, source.system_name
		, source.system_version
		, source.system_type
		, source.external_schema
		, source.is_production
	);

Generate schemas that will be used by the data warehouse.

1. dwh - This schema will contain tables and stored procedures related to the core data warehouse
2. master - This schema will contain tables and stored procedures used for holding normalized master data 
3. olap - This schema will contain views to expose data from the dwh and master schemas to Analysis Services
4. stage - This schema will contain tables used for storing data that needs to be staged before or during the ETL process.
5. Any schemas listed in the external_schema column in the [dbo].[SourceSystem] table.

In [ ]:
-- Set up temp table
IF OBJECT_ID('tempdb..#Temp') IS NOT NULL
	DROP TABLE #Temp;
GO

CREATE TABLE #Temp
(
	Id INT IDENTITY(1,1),
    SchemaName VARCHAR(32)
);
GO

-- Insert standard schemas
INSERT INTO #Temp (SchemaName) VALUES ('dwh'); --Data Warehouse Schema
INSERT INTO #Temp (SchemaName) VALUES ('master'); --Master table Schema
INSERT INTO #Temp (SchemaName) VALUES ('olap'); --SSAS View Schema
INSERT INTO #Temp (SchemaName) VALUES ('stage'); --Staging table Schema

-- Insert schemas from SourceSystem table
INSERT INTO #Temp (SchemaName)
SELECT DISTINCT external_schema
FROM [dbo].[SourceSystem];

-- Create schemas
DECLARE @counter INT = 1;
DECLARE @SchemaName VARCHAR(32) = 'dwh';
DECLARE @Sql NVARCHAR(128);

WHILE @counter <= (SELECT COUNT(1) FROM #Temp)
BEGIN
	SET @SchemaName = (SELECT SchemaName FROM #Temp WHERE Id = @counter);
	SET @counter = @counter + 1;
	SET @Sql = N'CREATE SCHEMA ' + @SchemaName;

	IF NOT EXISTS (
		SELECT  schema_name
		FROM    information_schema.schemata
		WHERE   schema_name = @SchemaName 
	) 
	BEGIN
	EXEC sp_executesql @Sql --ARCHIVE is the schema I want to create
	END
END
GO

Create standard stored procedures for importing data.

In [ ]:
DECLARE @Counter INT = 1;
DECLARE @SystemName VARCHAR(32);
DECLARE @ProcName AS NVARCHAR(MAX);
DECLARE @Sql AS varchar(MAX);

WHILE @counter <= (SELECT COUNT(1) FROM [dbo].[SourceSystem] WHERE source_system_id > 0)
BEGIN
	SET @SystemName = (SELECT system_name FROM [dbo].[SourceSystem] WHERE source_system_id = @counter);
	SET @counter = @counter + 1;
	SET @ProcName = '[dwh].[Import_' + REPLACE(@SystemName, ' ', '_') + ']';

	SET @Sql = 
	'
	CREATE PROCEDURE ' + @ProcName + '
	AS 
	BEGIN
		SET NOCOUNT ON;

		--INSERT CODE HERE
	END';

	IF NOT EXISTS ( 
		SELECT * 
		FROM   sysobjects 
		WHERE  id = object_id(@ProcName) and OBJECTPROPERTY(id, N'IsProcedure') = 1 
	)
	BEGIN
		EXEC(@Sql);
	END
END
GO

Create store procedure for creating a master date table.

In [ ]:
CREATE OR ALTER PROCEDURE [master].[Initialize_DateTable]
AS
BEGIN
	SET NOCOUNT ON;

	-- Set variables
	DECLARE @StartDate DATE  = '2011-01-01'; -- Start-date
	DECLARE @EndDate DATE =  CAST(YEAR(GETDATE()) + 2 AS CHAR(4)) + '-01-01'; -- End-Date
	   
	DECLARE @DefaultDate DATE = '1900-01-01';
	DECLARE @CurrentDate DATE;

	DECLARE @FiscalStartDate DATE = '2010-09-01';
	DECLARE @FiscalDifference INT = DATEDIFF(m, @FiscalStartDate, @StartDate);

	IF NOT EXISTS (SELECT * FROM sysobjects WHERE name= N'Holiday' AND XTYPE='U')
	BEGIN
		CREATE TABLE [master].[Holiday](
			[holiday_key] [nchar](18) NOT NULL,
			[actual_date] [datetime] NOT NULL,
			[observed_date] [datetime] NOT NULL,
			[description] [nvarchar](32) NULL,
			[created_date] [datetime] NOT NULL,
			[modified_date] [datetime] NULL,
			CONSTRAINT [PK_master_holiday] PRIMARY KEY CLUSTERED ([holiday_key] ASC)
		);
	END

	-- Drop table if it already exists
    IF EXISTS (SELECT * FROM sysobjects WHERE name= N'Date' AND XTYPE='U')
		DROP TABLE [master].[Date];

	-- Create new table
	CREATE TABLE [master].[Date](
		[FullDate] [date] NOT NULL,
		[DayOfWeek] [int] NOT NULL CONSTRAINT DF_Date_DayOfWeek DEFAULT 0,
		[DayOfWeekName] [varchar](10) NOT NULL CONSTRAINT DF_Date_DayOfWeekName DEFAULT '',
		[DayOfWeekAbbrev] [varchar](5) NOT NULL CONSTRAINT DF_Date_DayOfWeekAbbrev DEFAULT '',
		[DayOfMonth] [int] NOT NULL CONSTRAINT DF_Date_DayOfMonth DEFAULT 0,
		[DayOfQuarter] [int] NOT NULL CONSTRAINT DF_Date_DayOfQuarter DEFAULT 0,
		[DayOfYear] [int] NOT NULL CONSTRAINT DF_Date_DayOfYear DEFAULT 0,
		[WeekOfYear] [int] NOT NULL CONSTRAINT DF_Date_WeekOfYear DEFAULT 0,
		[MonthOfYear] [int] NOT NULL CONSTRAINT DF_Date_MonthOfYear DEFAULT 0,
		[MonthName] [varchar](10) NOT NULL CONSTRAINT DF_Date_MonthName DEFAULT '',
		[MonthNameAbbrev] [varchar](5) NOT NULL CONSTRAINT DF_Date_MonthNameAbbrev DEFAULT '',
		[QuarterOfYear] [int] NOT NULL CONSTRAINT DF_Date_QuarterOfYear DEFAULT 0,
		[Year] [int] NOT NULL CONSTRAINT DF_Date_Year DEFAULT 0,
		[FiscalDayOfMonth] [int] NOT NULL CONSTRAINT DF_Date_FiscalDayOfMonth DEFAULT 0,
		[FiscalDayOfQuarter] [int] NOT NULL CONSTRAINT DF_Date_FiscalDayOfQuarter DEFAULT 0,
		[FiscalDayOfYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalDayOfYear DEFAULT 0,
		[FiscalWeekOfYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalWeekOfYear DEFAULT 0,
		[FiscalMonthOfYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalMonthOfYear DEFAULT 0,
		[FiscalQuarterOfYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalQuarterOfYear DEFAULT 0,
		[FiscalYear] [int] NOT NULL CONSTRAINT DF_Date_FiscalYear DEFAULT 0,
		[IsBusinessDay] [bit] NOT NULL CONSTRAINT DF_Date_IsBusinessDay DEFAULT 0,
		[IsWeekday] [bit] NOT NULL CONSTRAINT DF_Date_IsWeekday DEFAULT 0,
		[IsHoliday] [bit] NOT NULL CONSTRAINT DF_Date_IsHoliday DEFAULT 0,
		[YYYYMM] [char](6) NOT NULL CONSTRAINT DF_Date_YYYYMM DEFAULT '',
		[YYYYMMDD] [char](8) NOT NULL CONSTRAINT DF_Date_YYYYMMDD DEFAULT '',
		[MMMYYYY] [char](9) NOT NULL CONSTRAINT DF_Date_MMMYYYY DEFAULT '',
		[MMYYYY] [char](7) NOT NULL CONSTRAINT DF_Date_MMYYYY DEFAULT '',
		[YYYYQQ] [char](7) NOT NULL CONSTRAINT DF_Date_YYYYQQ DEFAULT '',
		[YYYYFQ] [char](7) NOT NULL CONSTRAINT DF_Date_YYYYFQ DEFAULT '',
		CONSTRAINT [PK_master_date] PRIMARY KEY CLUSTERED ([FullDate] ASC)
	);

	-- Add default date 
	INSERT INTO [master].[Date] ([FullDate], [YYYYMM], [YYYYMMDD], [MMMYYYY], [MMYYYY], [YYYYQQ], [YYYYFQ])
	VALUES (@DefaultDate, '', '', '', '', '', '');

	-- Build new date table data
	INSERT INTO [master].[Date] (
		  [FullDate]
		, [DayOfWeek]
		, [DayOfWeekName]
		, [DayOfWeekAbbrev]
		, [DayOfMonth]
		, [DayOfQuarter]
		, [DayOfYear]
		, [WeekOfYear]
		, [MonthOfYear]
		, [MonthName]
		, [MonthNameAbbrev]
		, [QuarterOfYear]
		, [Year]
		, [FiscalDayOfMonth]
		, [FiscalDayOfQuarter]
		, [FiscalDayOfYear]
		, [FiscalWeekOfYear]
		, [FiscalMonthOfYear]
		, [FiscalQuarterOfYear]
		, [FiscalYear]
		, [IsBusinessDay]
		, [IsWeekday]
		, [IsHoliday]
		, [YYYYMM]
		, [YYYYMMDD]
		, [MMMYYYY]
		, [MMYYYY]
		, [YYYYQQ]
		, [YYYYFQ]
	)
		SELECT
		  a.CurrDate AS [FullDate]
		, DATEPART(DW, a.CurrDate) AS [DayOfWeek]
		, DATENAME(DW, a.CurrDate) AS [DayOfWeekName]
		, CASE WHEN DATEPART(DW, a.CurrDate) IN (3, 5) THEN LEFT(DATENAME(DW, a.CurrDate), 4)
			ELSE LEFT(DATENAME(DW, a.CurrDate), 3)
			END AS [DayOfWeekAbbrev]
		, DATEPART(DD, a.CurrDate) AS [DayOfMonth]
		, DATEDIFF(DD,
			DATEADD(qq, DATEDIFF(qq, 0, a.CurrDate), 0),
			a.CurrDate) + 1
			AS [DayOfQuarter]
		, DATEPART(DY, a.CurrDate) AS [DayOfYear]
		, DATEPART(WW, a.CurrDate) AS [WeekOfYear]
		, DATEPART(MM, a.CurrDate) AS [MonthOfYear]
		, DATENAME(MM, a.CurrDate) AS [MonthName]
		, CASE WHEN DATEPART(MM, a.CurrDate) = 9 THEN LEFT(DATENAME(MM, a.CurrDate), 4)
			ELSE LEFT(DATENAME(MM, a.CurrDate), 3)
			END AS [MonthNameAbbrev]
		, DATEPART(QQ, a.CurrDate) AS [QuarterOfYear]
		, DATEPART(YY, a.CurrDate) AS [Year]
		, DATEPART(DD, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalDayOfMonth]
		, DATEDIFF(DD,
			DATEADD(qq, DATEDIFF(qq, 0, DATEADD(m, @FiscalDifference, a.CurrDate)), 0),
			DATEADD(m, @FiscalDifference, a.CurrDate)) + 1
			AS [FiscalDayOfQuarter]
		, DATEPART(DY, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalDayOfYear]
		, DATEPART(WW, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalWeekOfYear]
		, DATEPART(MM, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalMonthOfYear]
		, DATEPART(QQ, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalQuarterOfYear]
		, DATEPART(YY, DATEADD(m, @FiscalDifference, a.CurrDate)) AS [FiscalYear]
		, CASE WHEN DATEPART(DW, a.CurrDate) = 1 OR DATEPART(DW, a.CurrDate) = 7 OR h.[holiday_key] IS NOT NULL THEN 0
			ELSE 1
			END AS [IsBusinessDay]
		, CASE WHEN DATEPART(DW, a.CurrDate) = 1 OR DATEPART(DW, a.CurrDate) = 7 THEN 0
			ELSE 1
			END AS [IsWeekday]
		, CASE WHEN h.[holiday_key] IS NULL THEN 0
			ELSE 1
			END AS [IsHoliday]
		, LEFT(CONVERT(VARCHAR, a.CurrDate, 112), 6) AS [YYYYMM]
		, CONVERT(VARCHAR, a.CurrDate, 112) AS [YYYYMMDD]
		, CASE WHEN DATEPART(MM, a.CurrDate) = 9 THEN LEFT(DATENAME(MM, a.CurrDate), 4) ELSE LEFT(DATENAME(MM, a.CurrDate), 3) END + ' ' + CAST(DATEPART(YY, a.CurrDate) AS char(4)) AS [MMMYYYY]
		, CASE WHEN MONTH(a.CurrDate) < 10 THEN '0' ELSE '' END + CAST(DATEPART(MM, a.CurrDate) AS varchar(4)) + '-' + CAST(DATEPART(YY, a.CurrDate) AS char(4)) AS [MMYYYY]
		, CAST(DATEPART(YY, a.CurrDate) AS char(4)) + '/Q' + CAST(DATEPART(QQ, a.CurrDate) AS char(1)) AS [YYYYQQ]
		, CAST(DATEPART(YY, DATEADD(m, @FiscalDifference, a.CurrDate)) AS char(4)) + '/Q' + CAST(DATEPART(QQ, DATEADD(m, @FiscalDifference, a.CurrDate)) AS char(1)) AS [YYYYFQ]
	FROM (
		SELECT CurrDate = CAST(DATEADD(DAY, rn - 1, @StartDate) AS DATE)
		FROM (
			SELECT TOP (DATEDIFF(DAY, @StartDate, @EndDate)) rn = ROW_NUMBER() OVER (ORDER BY s1.[object_id])
			FROM sys.all_objects AS s1
				CROSS JOIN sys.all_objects AS s2
		    ORDER BY s1.[object_id]) AS x
	) a
		LEFT OUTER JOIN [master].[Holiday] h ON a.CurrDate = h.observed_date;

END
GO

Run the stored procedure to create the new DateTable.

In [ ]:
EXEC [master].[Initialize_DateTable];
GO

Create stored procedure to call for importing data from the data warehouse layer to the master layer.

In [ ]:
DECLARE @ProcName AS NVARCHAR(MAX);
DECLARE @Sql AS varchar(MAX);

SET @ProcName = '[master].[Import_Warehouse]';

SET @Sql = 
'
CREATE PROCEDURE ' + @ProcName + '
AS 
BEGIN
	SET NOCOUNT ON;

	--INSERT CODE HERE
END';

IF NOT EXISTS ( 
	SELECT * 
	FROM   sysobjects 
	WHERE  id = object_id(@ProcName) and OBJECTPROPERTY(id, N'IsProcedure') = 1 
)
BEGIN
	EXEC(@Sql);
END

Create stored procedure used for normalizing data.

In [ ]:
DECLARE @ProcName AS NVARCHAR(MAX);
DECLARE @Sql AS varchar(MAX);

SET @ProcName = '[dwh].[Normalize_Data]';

SET @Sql = 
'
CREATE PROCEDURE ' + @ProcName + '
AS 
BEGIN
	SET NOCOUNT ON;

	--INSERT CODE HERE
END';

IF NOT EXISTS ( 
	SELECT * 
	FROM   sysobjects 
	WHERE  id = object_id(@ProcName) and OBJECTPROPERTY(id, N'IsProcedure') = 1 
)
BEGIN
	EXEC(@Sql);
END

For error handling, see script "Error Logging - MSSQL".